In [1]:
import os
import sys

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

import numpy as np
import open3d as o3d
import torch
import matplotlib.pyplot as plt


# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Import custom modules (ensure the paths are correct)
from src.utils.data_utils import load_point_cloud, save_point_cloud
#from src.data_processing.data_augmentation import add_random_noise_points
from src.data_processing.data_augmentation import add_random_noise_points
from src.data_processing.data_preprocessing import (
    voxel_down_sample_with_indices,
    adjust_point_count_with_indices,
    normalize_point_cloud,
)

from src.models.pointnetplusplus import PointNetPlusPlus
from src.models.pointnet2_utils import PointNetSetAbstraction, PointNetSetAbstractionMsg, PointNetFeaturePropagation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Specify the path to a sample point cloud file
sample_ply = r'C:\Users\joe_h\Desktop\work_projects\Plant-Point-Cloud-Filtering-with-Supervised-Segmentation\data\raw\plant_only\Wheat_Gladius_B6_2023-06-27-2029_fused_output.ply'  # Replace with an actual filename

# Load the point cloud
points, colors = load_point_cloud(sample_ply)

# Create an Open3D point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
if colors is not None:
    pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
print("Original Point Cloud:")
#o3d.visualization.draw_geometries([pcd])

Original Point Cloud:


In [3]:
# Number of noise points to add
num_noise_points = 5000  # Adjust as needed

# Call the function to get the combined point cloud and colors

combined_points, combined_colors, labels = add_random_noise_points(points, colors, num_noise_points)

# After adding noise
print(f"Total points in combined point cloud: {len(combined_points)}")
print(f"Number of plant points: {np.sum(labels == 1)}")
print(f"Number of noise points: {np.sum(labels == 0)}")

Total points in combined point cloud: 75224
Number of plant points: 70224
Number of noise points: 5000


In [4]:
# Convert to Open3D point cloud for processing
noisy_pcd = o3d.geometry.PointCloud()
noisy_pcd.points = o3d.utility.Vector3dVector(combined_points)
noisy_pcd.colors = o3d.utility.Vector3dVector(combined_colors)

# Downsample the point cloud
voxel_size = 0.005  # Adjust as needed

# Extract indices of the downsampled points
# Downsample the point cloud
downsampled_pcd, downsampled_indices = voxel_down_sample_with_indices(noisy_pcd, voxel_size)

# Adjust the labels based on the downsampled indices
downsampled_labels = labels[downsampled_indices]

print(f"Downsampled point cloud has {len(downsampled_pcd.points)} points")
print(f"Number of plant points: {np.sum(downsampled_labels == 1)}")
print(f"Number of noise points: {np.sum(downsampled_labels == 0)}")

# Adjust the point count
num_points = 2048


Downsampled point cloud has 12279 points
Number of plant points: 7291
Number of noise points: 4988


In [5]:
# Use the modified function
adjusted_pcd, adjusted_labels = adjust_point_count_with_indices(downsampled_pcd, downsampled_labels, num_points)

# Verify the counts
print(f"Adjusted point cloud has {len(adjusted_pcd.points)} points")
print(f"Number of plant points: {np.sum(adjusted_labels == 1)}")
print(f"Number of noise points: {np.sum(adjusted_labels == 0)}")


Adjusted point cloud has 2048 points
Number of plant points: 1217
Number of noise points: 831


In [6]:
o3d.visualization.draw_geometries([adjusted_pcd])

In [6]:
# Step 4: Normalize the point cloud
normalized_pcd = normalize_point_cloud(adjusted_pcd)
normalized_points = np.asarray(normalized_pcd.points)
normalized_colors = np.asarray(normalized_pcd.colors)

# Step 5: Prepare input tensors
input_points = torch.from_numpy(normalized_points).float().unsqueeze(0)  # Shape: (1, N, 3)
input_labels = torch.from_numpy(adjusted_labels).long().unsqueeze(0)     # Shape: (1, N)

print(f"Input points shape: {input_points.shape}")
print(f"Input labels shape: {input_labels.shape}")
print(f"Number of plant points: {np.sum(adjusted_labels == 1)}")
print(f"Number of noise points: {np.sum(adjusted_labels == 0)}")

Input points shape: torch.Size([1, 2048, 3])
Input labels shape: torch.Size([1, 2048])
Number of plant points: 1217
Number of noise points: 831


In [7]:
# Initialize the PointNet++ model
num_classes = 2  # For binary segmentation
model = PointNetPlusPlus(num_classes=num_classes)

# Set the model to evaluation mode
model.eval()

# Forward pass
with torch.no_grad():
    output = model(input_points)  # Output shape: (1, N, num_classes)
    predicted_labels = torch.argmax(output, dim=2)  # Shape: (1, N)
    predicted_labels_np = predicted_labels.squeeze().numpy()  # Shape: (N,)

print(f"Output tensor shape: {output.shape}")
print(f"Predicted labels shape: {predicted_labels.shape}")
print(f"Unique predicted labels: {np.unique(predicted_labels_np)}")

Output tensor shape: torch.Size([1, 2048, 2])
Predicted labels shape: torch.Size([1, 2048])
Unique predicted labels: [0]
